In [ ]:
import numpy as np
import torch
import torchvision
import os
import math
import matplotlib.pyplot as plt
from torchvision import datasets, models, transforms
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, Dataset
from math import ceil
import torch.nn as nn
import torch.optim as optim
import time
import torch.nn.functional as F
import csv
from torch.optim import lr_scheduler
from torch.autograd import gradcheck

In [ ]:
ROOT_DIR = './Assignments/Assignment 1/ift6135h19'

BATCH_SIZE = 32

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
# data_loader

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomRotation(10),
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop(64, scale=(0.80, 1.0)),
        transforms.ColorJitter(),
        transforms.RandomGrayscale(0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Device: {}'.format(device))


def data_loader(train_to_valid_ratio=0.8, 
                root_dir=ROOT_DIR, 
                batch_size=BATCH_SIZE):

    train_valid_data = ImageFolderWithPaths(
        os.path.join(root_dir, 'trainset'), data_transforms['train'])
    
    test_data = ImageFolderWithPaths(os.path.join(
        root_dir, 'testset'), data_transforms['test'])
    
    print(type(train_valid_data))
    
    print('class_to_idx: {}'.format(train_valid_data.class_to_idx))
    class_to_idx = train_valid_data.class_to_idx
    idx_to_class = {v: k for k, v in class_to_idx.items()}
    
    print('Channels in train_valid_data[0]: {}\n'.format(len(train_valid_data[0][0].shape)))
    
    train_valid_data_size = len(train_valid_data)
    print(type(train_valid_data))
    train_valid_indices = list(range(train_valid_data_size))
    split = int(np.ceil(train_to_valid_ratio * train_valid_data_size))
    print('split = {}'.format(split))
    
    print('Image size = {}, Label = {}\n'.format(train_valid_data[0][0].shape, train_valid_data[0][1]))

    # shuffle the indices
    np.random.shuffle(train_valid_indices)
    train_indices, valid_indices = train_valid_indices[:split], train_valid_indices[split:]
    
    print('len(train_indices): {}'.format(len(train_indices)))
    print('len(valid_indices): {}'.format(len(valid_indices)))
    
    print('Size of test data: {}'.format(len(test_data)))

    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(valid_indices)

    train_loader = DataLoader(train_valid_data, batch_size=batch_size, sampler=train_sampler)
    valid_loader = DataLoader(train_valid_data, batch_size=batch_size, sampler=valid_sampler)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    return (train_loader, valid_loader, test_loader, 
            class_to_idx, idx_to_class)


In [ ]:
train_loader, valid_loader, test_loader, class_to_idx, idx_to_class = data_loader(train_to_valid_ratio=0.85)


In [ ]:
from torch.optim.sgd import SGD
from torch.optim.optimizer import required

class SGDCustom(SGD):
    def __init__(self, params, lr=required, momentum=0, dampening=0, weight_decay=0, nesterov=False):
        kwargs = dict(lr=lr, momentum=momentum, dampening=dampening, 
                      weight_decay=weight_decay, nesterov=nesterov)
        super().__init__(params, **kwargs)

    def step(self, closure=None):
        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data
                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data) 
                d_p.mul_(group['lr'])
                if momentum != 0:
                    param_state = self.state[p]
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.zeros_like(p.data)
                        buf.mul_(momentum).add_(d_p)
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                    d_p = buf

                p.data.add_(-1, d_p)

        return loss

## Define VGG Model

In [ ]:
# # VGG13 - VGG12 - [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

class ConvNetD_withDropOut_VGG(nn.Module):
    """Convnet Classifier"""

    def __init__(self, dropout_prob=0.0, print_dropout=True):
        super(ConvNetD_withDropOut_VGG, self).__init__()
        self.print_dropout = print_dropout
        self.dropout_prob = float(dropout_prob)
        
        self.conv_1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3, 3), padding=1) # 64*64*3, 32*32*64
        torch.nn.init.xavier_uniform_(self.conv_1.weight)
        
        self.conv_2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding=1) # 32*32*64, 16*16*128
        torch.nn.init.xavier_uniform_(self.conv_2.weight)
        
        self.conv_3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3, 3), padding=1) #16*16*128, 16*16*256
        torch.nn.init.xavier_uniform_(self.conv_3.weight)
        
        self.conv_4 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3, 3), padding=1) #16*16*256, 8*8*256
        torch.nn.init.xavier_uniform_(self.conv_4.weight)
        
        self.conv_5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3, 3), padding=1) #8*8*256, 8*8*512
        torch.nn.init.xavier_uniform_(self.conv_5.weight)
        
        self.conv_6 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1) #8*8*512, 4*4*512
        torch.nn.init.xavier_uniform_(self.conv_6.weight)
        
        self.conv_7 = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3, 3), padding=1) #4*4*512, 4*4*512
        torch.nn.init.xavier_uniform_(self.conv_7.weight)
                
        
        self.fc_1 = nn.Linear(4*4*512, 4096)
        torch.nn.init.xavier_uniform_(self.fc_1.weight)
        self.fc_2 = nn.Linear(4096, 4096)
        torch.nn.init.xavier_uniform_(self.fc_2.weight)
        self.fc_3 = nn.Linear(4096, 2)
        torch.nn.init.xavier_uniform_(self.fc_3.weight)


    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv_1(x)), kernel_size=(2, 2), stride=2)
        x = F.max_pool2d(F.relu(self.conv_2(x)), kernel_size=(2, 2), stride=2)
        x = F.relu(self.conv_3(x))
        x = F.max_pool2d(F.relu(self.conv_4(x)), kernel_size=(2, 2), stride=2)
        x = F.relu(self.conv_5(x))
        x = F.max_pool2d(F.relu(self.conv_6(x)), kernel_size=(2, 2), stride=2)
        x = F.relu(self.conv_7(x))
        
        x = x.view(-1, 4*4*512)
        
        x = F.relu(self.fc_1(x))
        if self.dropout_prob > 0.0 and self.training:
            if self.print_dropout:
                print('Using dropout: {} at fc_1'.format(self.dropout_prob))
            dropout_1 = ((torch.rand(x.size()) < self.dropout_prob).float() / self.dropout_prob).float().cuda()
            x *= dropout_1
        
        x = F.relu(self.fc_2(x))
        if self.dropout_prob > 0.0 and self.training:
            if self.print_dropout:
                print('Using dropout: {} at fc_2'.format(self.dropout_prob))
                self.print_dropout = False
            dropout_2 = ((torch.rand(x.size()) < self.dropout_prob).float() / self.dropout_prob).float().cuda()
            x *= dropout_2
            
        x = self.fc_3(x)
        
        return x

In [ ]:
decoder_attempt_withDropOut_VGG_customSGD_ES = ConvNetD_withDropOut_VGG(0.6)
decoder_attempt_withDropOut_VGG_customSGD_ES.cuda()

print(decoder_attempt_withDropOut_VGG_customSGD_ES.parameters)

optimizer = SGDCustom(decoder_attempt_withDropOut_VGG_customSGD_ES.parameters(), 
                        lr=0.01,momentum=0.9, weight_decay=1.5e-3) 

scheduler_plateau = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.05, patience=10, 
                  verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

criterion = nn.CrossEntropyLoss()

start_time = time.time()

best_loss = math.inf
best_accuracy = 0.0

n_epochs = 100

count = 0

for epoch in range(n_epochs):
    print('\n', '-'*10, ' Epoch: {} '.format(epoch), '-'*10)

    decoder_attempt_withDropOut_VGG_customSGD_ES.train() # set training mode
    
    train_dataset_size = 0
    correct_train_preds = 0
    running_train_loss = 0.0
    
    # training
    for inputs, labels, ids in train_loader:
        random_perm = torch.randperm(len(inputs))
        inputs = inputs[random_perm]
        labels = labels[random_perm]
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # forward prop
        outputs = decoder_attempt_withDropOut_VGG_customSGD_ES(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        
        # backprop
        loss.backward()
        optimizer.step()
        
        # metrics
        running_train_loss += loss.item() * inputs.size(0)
        correct_train_preds += torch.sum(preds == labels.data).item()
        train_dataset_size += len(labels)
        
        
    epoch_train_loss = running_train_loss / train_dataset_size
    epoch_train_accuracy = (correct_train_preds*1.0) / train_dataset_size 
    
    print('Training Loss: {}, Accuracy: {}'.format(epoch_train_loss, epoch_train_accuracy))
    
    
    # validation
    
    decoder_attempt_withDropOut_VGG_customSGD_ES.eval() # set eval mode
    
    valid_dataset_size = 0
    correct_valid_preds = 0
    running_valid_loss = 0.0
    
    for inputs, labels, ids in valid_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = decoder_attempt_withDropOut_VGG_customSGD_ES(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        
        running_valid_loss += loss.item() * inputs.size(0)
        correct_valid_preds += torch.sum(preds == labels.data).item()
        valid_dataset_size += len(labels)

    epoch_valid_loss = running_valid_loss / valid_dataset_size
    epoch_valid_accuracy = (correct_valid_preds*1.0) / valid_dataset_size
    
    decoder_attempt_withDropOut_VGG_customSGD_ES.train()
    
    if (epoch_valid_accuracy - best_accuracy) > 0.0001:
        best_accuracy = epoch_valid_accuracy
        torch.save(decoder_attempt_withDropOut_VGG_customSGD_ES.state_dict(), 'saved_model')
        
    
    print('Validation Loss: {}, Accuracy: {}'.format(epoch_valid_loss, epoch_valid_accuracy))
    
    scheduler_plateau.step(epoch_valid_loss)
        
        
time_elapsed = time.time() - start_time

print('Training completed in {}minutes {}secs'.format(time_elapsed // 60, time_elapsed % 60))


In [ ]:
def save_predictions(model, outfile_name):
    
    model.eval()
    
    ids_and_predictions = dict()
    
    for inputs, labels, ids in test_loader:
        ids_list = [x.split('/')[-1].split('.')[0] for x in ids]
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        
        preds_list = list(preds.cpu().numpy())
        
        count = 0

        for (idx, pred) in zip(ids_list, preds_list):
            idx = int(idx)
            ids_and_predictions[idx] = idx_to_class[pred]
        
    print(len(ids_and_predictions))
        
    with open(outfile_name + '.csv', 'w') as out_file:
        csv_writer = csv.writer(out_file)
        csv_writer.writerow(['id', 'label'])
        for idx in range(1, len(ids_and_predictions) + 1):
            csv_writer.writerow([idx, ids_and_predictions[idx]])

In [ ]:
save_predictions(decoder_attempt_withDropOut_VGG_customSGD_ES, 
                 'trained_model')